# Table tennis simulation

This example shows the usage of `Simulate.jl` with event driven state machines.

In [1]:
using Simulate, Printf
import Simulate.init!

Here we implement players as timed state machines and thus need some data describing the players and definitions of states and events:

In [2]:
abstract type PState end
struct Idle <: PState end
struct Wait <: PState end
struct Unalert <: PState end

abstract type PEvent end
struct Start <: PEvent end
struct Serve <: PEvent end
struct Return <: PEvent end
struct Miss <: PEvent end

mutable struct Player
    name::AbstractString
    opp::Union{Number,Player}
    state::PState
    accuracy::Float64
    attentiveness::Float64
    score::Int64

    Player(name, acc, att) = new(name, 0, Idle(), acc, att, 0)
end

Then we have some physical facts to define and a function to randomize them:

In [3]:
const dist = 3 # distance for ball to fly [m]
const vs   = 10 # serve velocity [m/s]
const vr   = 20 # return velocity [m/s]

rd(s::Float64) = randn()*s + 1

rd (generic function with 1 method)

Next we must describe the behaviour of our players. They are modeled as finite state machines, which have known states and react to known events. This is done with the `step!` function. Julia's multiple dispatch allows to give multiple definitions of `step!` for different combinations of states and events.

The `serve` and `ret`-functions, used for describing serving and return of players are used to randomize the time and the behaviour of players. The players thus act probabilistically as Markov automata.

The `serve`und `ret`-functions use some `Sim.jl`-features:

- `𝐶` (`\itC`+Tab) or `Clk` is the central simulation clock, 
- `τ()` or `tau()` gives the time of this clock,
- `event!` schedules an expression or a function after some time. In this case we use expressions for illustration. (Normally it is easier to schedule functions with `SimFunction`.)
- in an expression we may interpolate some variables, e.g. `$(p.opp)`, which includes the variable content, not the symbol in the expression. 

In [4]:
function init!(p::Player, opp::Player)
    p.opp = opp
    if rand() ≤ p.attentiveness
        p.state = Wait()
    else
        p.state = Unalert()
    end
end

function serve(p::Player)
    ts = 3 + dist*rd(0.15)/(vs*rd(0.25))
    if (rand() ≤ p.accuracy) && (p.state == Wait())
        event!(𝐶, :(step!($(p.opp), Serve())), after, ts)
        @printf("%.2f: %s serves %s\n", τ()+ts, p.name, p.opp.name)
    else
        event!(𝐶, :(step!($(p.opp), Miss())), after, ts)
        @printf("%.2f: %s serves and misses %s\n", τ()+ts, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

function ret(p::Player)
    tr = dist*rd(0.15)/(vr*rd(0.25))
    if rand() ≤ p.accuracy
        event!(𝐶, :(step!($(p.opp), Return())), after, tr)
        @printf("%.2f: %s returns %s\n", τ()+tr, p.name, p.opp.name)
    else
        event!(𝐶, :(step!($(p.opp), Miss())), after, tr)
        @printf("%.2f: %s returns and misses %s\n", τ()+tr, p.name, p.opp.name)
    end
    if rand() ≥ p.attentiveness
        p.state = Unalert()
    end
end

ret (generic function with 1 method)

The actual behaviour of a player is implemented as a state machine via the `step!`-$\delta$-function. Here we implement different methods $\delta(p, q, \sigma)$ which are called at runtime according to states $q$ ond events $\sigma$:

In [5]:
"default transition for players"
step!(p::Player, q::PState, σ::PEvent) =
        println("undefined transition for $(p.name), $q, $σ")

"player p gets a start command"
step!(p::Player, ::Union{Wait, Unalert}, ::Start) = serve(p)

"player p is waiting and gets served or returned"
step!(p::Player, ::Wait, ::Union{Serve, Return}) = ret(p)

"player p is unalert and gets served or returned"
function step!(p::Player, ::Unalert, ::Union{Serve, Return})
    @printf("%.2f: %s looses ball\n", τ(), p.name)
    p.opp.score += 1
    p.state = Wait()
    serve(p)
end

"player p is waiting or unalert and gets missed"
function step!(p::Player, ::Union{Wait, Unalert}, ::Miss)
    p.score += 1
    p.state = Wait()
    serve(p)
end

"simplified `step!` call"
step!(p::Player, σ::PEvent) = step!(p, p.state, σ)

step!

In order to setup a simulation, we have to create and initialize the players, to start and run the game:

In [6]:
ping = Player("Ping", 0.90, 0.90)
pong = Player("Pong", 0.90, 0.90)
init!(ping, pong)
init!(pong, ping)
step!(ping, Start())

run!(𝐶, 30)
println("Ping scored $(ping.score)")
println("Pong scored $(pong.score)")

3.42: Ping serves Pong
3.55: Pong returns Ping
3.65: Ping returns Pong
3.80: Pong returns Ping
3.96: Ping returns and misses Pong
7.27: Pong serves Ping
7.39: Ping returns and misses Pong
10.68: Pong serves Ping
10.89: Ping returns Pong
11.41: Pong returns Ping
11.53: Ping returns Pong
11.86: Pong returns Ping
11.99: Ping returns and misses Pong
15.26: Pong serves Ping
15.39: Ping returns Pong
15.49: Pong returns Ping
15.60: Ping returns Pong
15.78: Pong returns Ping
15.91: Ping returns Pong
16.05: Pong returns and misses Ping
19.39: Ping serves Pong
19.54: Pong returns Ping
19.54: Ping looses ball
22.80: Ping serves and misses Pong
26.24: Pong serves Ping
26.24: Ping looses ball
29.76: Ping serves Pong
29.76: Pong looses ball
33.15: Pong serves Ping
Ping scored 2
Pong scored 6


Finally we should reset `𝐶` for further simulations.

In [7]:
reset!(𝐶)

"clock reset to t₀=0.0, sampling rate Δt=0.0."